In [1]:
import numpy as np
import tensorflow.contrib.keras as kr
import torch
from torch import nn
from model import TextRNN
from cnews_loader import read_vocab,read_category, process_file
from torch import optim
import torch.utils.data as Data

C:\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:529: Futu

['体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐']
x_train= [[1609  659   56 ...    9  311    3]
 [   2  101   16 ... 1168    3   24]
 [ 465  855  521 ...  116  136   85]
 ...
 [  49   18   79 ...  836 1928 1072]
 [ 166  110  714 ...  836 1928 1072]
 [   1   80  551 ...   78  192    3]]


In [2]:
# 数据目标
train_file ='cnews.train.txt'
test_file ='cnews.test.txt'
val_file ='cnews.val.txt'
vocab_file ='cnews.vocab.txt'

In [3]:
def train():
    model = TextRNN()
    Loss = nn.MultiLabelSoftMarginLoss
    optimizer = optim.Adam(model.parameters(),lr=0.001)
    # 分批
    best_val_acc = 0
    for epoch in range(20):
        print('epoch = ',epoch )
        for step,(x_batch,y_batch) in enumerate(train_loader):
            x = x_batch
            y = y_batch
            out = model(x)
            loss = Loss(out,y)
            print('loss = ',loss)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            accuracy = np.mean((torch.argmax(out,1) == torch.argmax(y,1)).cpu().numpy())
            print('accuracy = :',accuracy)
            
        if (epoch+1) % 5 ==0:
            for step,(x_batch,y_batch) in enumerate(train_loader):
                x = x_batch
                y = y_batch
                out = model(x)
                accuracy = np.mean((torch.argmax(out,1) == torch.argmax(y,1)).cpu().numpy())
                if accuracy > best_val_acc:
                    torch.save(model,'model_pkl')
                    best_val_acc = accuracy
                    print(model.saved)
                    print('accuracy = : ',accuracy)


In [4]:
# 预处理
categories, cat_to_id = read_category()
print(categories)

# 字及对应的id
words, word_to_id = read_vocab('cnews.vocab.txt')
vocab_size = len(words)

# 分批训练;每个字的id和对应标签(one-hot编码)
x_train, y_train = process_file('cnews.train.txt', word_to_id, cat_to_id, 600)
print('x_train=', x_train)
x_val, y_val = process_file('cnews.val.txt', word_to_id, cat_to_id, 600)


['体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐']
x_train= [[1609  659   56 ...    9  311    3]
 [   2  101   16 ... 1168    3   24]
 [ 465  855  521 ...  116  136   85]
 ...
 [  49   18   79 ...  836 1928 1072]
 [ 166  110  714 ...  836 1928 1072]
 [   1   80  551 ...   78  192    3]]


In [5]:
# 转换数据格式
x_train,y_train = torch.LongTensor(x_train),torch.Tensor(y_train)
x_val,y_val = torch.LongTensor(x_val),torch.Tensor(y_val)

# 训练集
train_dataset = Data.TensorDataset(x_train,y_train)
train_loader = Data.DataLoader(dataset = train_dataset,batch_size = 32 ,shuffle = True)

# 验证集
val_dataset = Data.TensorDataset(x_val,y_val)
val_loader = Data.DataLoader(dataset =val_dataset,batch_size = 32)

In [6]:
# 训练
train()

epoch =  0


C:\Anaconda3\envs\tensorflow\lib\site-packages\torch\nn\modules\container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


RuntimeError: Boolean value of Tensor with more than one value is ambiguous